In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


In [86]:
import cv2, os
import numpy as np
from tqdm import tqdm
from services.semantic_keypoint_detection.services import DATATYPES

orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
sift = cv2.SIFT_create()

cache_orb_descriptors = []
cache_sift_descriptors = []

for i in range(8):
    cache_orb_descriptors.append([])
    cache_sift_descriptors.append([])

    for j in tqdm(range(500), f"Cache data of type {i}"):
        image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{j}.png"))
        gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ground_truth = np.load(os.path.join(DATATYPES[i], "points", f"{j}.npy"))
        ground_truth = [cv2.KeyPoint(y, x, 1) for x, y in ground_truth]

        orb_descriptors = orb.compute(gray_scale, ground_truth)[1]
        sift_descriptors = sift.compute(gray_scale, ground_truth)[1]

        cache_orb_descriptors[i].append(orb_descriptors)
        cache_sift_descriptors[i].append(sift_descriptors)

Cache data of type 0:   0%|          | 0/500 [00:00<?, ?it/s]

Cache data of type 7: 100%|██████████| 500/500 [00:00<00:00, 654.64it/s]


In [99]:
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
sift = cv2.SIFT_create()

similarity_orb_rotation = []
similarity_sift_rotation = []

for alpha in range(0, 360, 10):
    similarity_orb = []
    similarity_sift = []

    for i in range(8):
        similarity_orb.append([])
        similarity_sift.append([])

        for j in tqdm(
            range(500), f"Caculate similarity of type {i} with alpha {alpha}"
        ):
            # Read image and rotate it
            image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{j}.png"))
            h, w = image.shape[:2]
            matrix_rotation = cv2.getRotationMatrix2D(
                (image.shape[1] / 2, image.shape[0] / 2), alpha, 1
            )
            rotated_image = cv2.warpAffine(
                image, matrix_rotation, (image.shape[1], image.shape[0])
            )
            gray_scale = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)

            # Get ground truth and rotate it
            ground_truth = np.load(os.path.join(DATATYPES[i], "points", f"{j}.npy"))
            if len(ground_truth) > 0:
                ground_truth = np.array([[y, x] for x, y in ground_truth])
                ground_truth_X = np.concatenate(
                    (ground_truth, np.ones((ground_truth.shape[0], 1))), axis=1
                ).T

                indices = []
                ground_truth_rotated = (matrix_rotation @ ground_truth_X).T
                for id, (x, y) in enumerate(ground_truth_rotated):
                    if 0 <= x < w and 0 <= y < h:
                        indices.append(id)

                if len(indices) > 0:
                    sift_descriptors = np.array(cache_sift_descriptors[i][j])[indices]
                    sift_descriptors_rotated = sift.compute(
                        gray_scale,
                        [
                            cv2.KeyPoint(x, y, 1)
                            for x, y in ground_truth_rotated[indices]
                        ],
                    )[1]

                    sift_cosine_similarity = 0
                    for descriptor, descriptor_rotated in zip(
                        sift_descriptors, sift_descriptors_rotated
                    ):
                        sift_cosine_similarity += cosine_similarity(
                            [descriptor], [descriptor_rotated]
                        )[0][0]
                    similarity_sift[i].append(sift_cosine_similarity / len(indices))

                    orb_descriptors = np.array(cache_orb_descriptors[i][j])[indices]
                    orb_descriptors_rotated = orb.compute(
                        gray_scale,
                        [
                            cv2.KeyPoint(x, y, 1)
                            for x, y in ground_truth_rotated[indices]
                        ],
                    )[1]

                    orb_cosine_similarity = 0
                    for descriptor, descriptor_rotated in zip(
                        orb_descriptors, orb_descriptors_rotated
                    ):
                        orb_cosine_similarity += cosine_similarity(
                            [descriptor], [descriptor_rotated]
                        )[0][0]
                    similarity_orb[i].append(orb_cosine_similarity / len(indices))

    similarity_orb_rotation.append(similarity_orb)
    similarity_sift_rotation.append(similarity_sift)

    sum_1, sum_2 = 0, 0
    for i in range(8):
        if len(similarity_orb[i]) == 0:
            continue
        sum_1 += np.mean(similarity_orb[i])
        sum_2 += np.mean(similarity_sift[i])

    print(f"Alpha: {alpha}, ORB: {sum_1 / 7}, SIFT: {sum_2 / 7}")

Caculate similarity of type 0 with alpha 0:   0%|          | 0/500 [00:00<?, ?it/s]

Caculate similarity of type 7 with alpha 0: 100%|██████████| 500/500 [00:01<00:00, 388.86it/s]


Alpha: 0, ORB: 0.9998736895288619, SIFT: 0.999873697757721


Caculate similarity of type 7 with alpha 10: 100%|██████████| 500/500 [00:23<00:00, 21.51it/s] 


Alpha: 10, ORB: 0.9117091660386952, SIFT: 0.8884617686271667


...

Alpha: 250, ORB: 0.7014002122969121, SIFT: 0.10797320306301117


Caculate similarity of type 7 with alpha 260: 100%|██████████| 500/500 [00:01<00:00, 446.51it/s]


Alpha: 260, ORB: 0.7004741383263003, SIFT: 0.11591864377260208


Caculate similarity of type 0 with alpha 270:  23%|██▎       | 116/500 [00:01<00:04, 95.20it/s] 


KeyboardInterrupt: 